<a href="https://colab.research.google.com/github/MoBanerjee/SC4001_Project/blob/main/EfficientNetb0_FullFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objective of Notebook
### This notebook fine-tunes all layers of a pre-trained EfficientNet-B0 model on the Flowers102 dataset. After loading and preparing the dataset, we unfreeze all layers for full fine-tuning, train the model with early stopping, and evaluate its performance on the test set, visualizing metrics to assess accuracy and loss trends.


In [ ]:
# necessary imports
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

In [ ]:
transform = transforms.Compose([
    # resize the image to 224x224 pixels (width, height)
    transforms.Resize((224, 224)),
    
    # convert the image to a PyTorch tensor (scales pixel values between 0 and 1)
    transforms.ToTensor(),
    
    # normalize the image using the mean and standard deviation values for each RGB channel
    # (these are typical values used for pre-trained models)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# load the training dataset of the Flowers102 dataset, applying the specified transformations
train_data = torchvision.datasets.Flowers102(
    root='./data',         # directory to store the dataset
    split='train',         # specifies that this is the training split
    transform=transform,   # apply the transformations defined earlier
    download=True          # download the dataset if it is not already present
)

# load the validation dataset of the Flowers102 dataset, applying the same transformations
# and downloading if necessary.
val_data = torchvision.datasets.Flowers102(
    root='./data',         # directory to store the dataset
    split='val',           # specifies that this is the validation split
    transform=transform,   # apply the transformations defined earlier
    download=True          # download the dataset if it is not already present
)

# load the test dataset of the Flowers102 dataset, applying the same transformations
# and downloading if necessary.
test_data = torchvision.datasets.Flowers102(
    root='./data',         # directory to store the dataset
    split='test',          # specifies that this is the test split
    transform=transform,   # apply the transformations defined earlier
    download=True          # download the dataset if it is not already present
)


100%|██████████| 345M/345M [00:13<00:00, 24.9MB/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 1.39MB/s]


100%|██████████| 15.0k/15.0k [00:00<00:00, 14.8MB/s]


In [ ]:
# create a DataLoader for the training dataset with a batch size of 32, shuffling the data
# to ensure that the model sees the data in a different order each epoch
train_loader = DataLoader(
    train_data,     # the training dataset
    batch_size=32,  # number of samples per batch
    shuffle=True    # shuffle the data for training
)

# create a DataLoader for the validation dataset with a batch size of 32
# shuffling is set to False since validation data should be consistent across epochs
val_loader = DataLoader(
    val_data,       # the validation dataset
    batch_size=32,  # number of samples per batch
    shuffle=False   # do not shuffle for validation
)

# create a DataLoader for the test dataset with a batch size of 32
# shuffling is set to False for consistent evaluation
test_loader = DataLoader(
    test_data,      # the test dataset
    batch_size=32,  # number of samples per batch
    shuffle=False   # do not shuffle for testing
)


In [ ]:
# load pre-trained EfficientNet-B0 model. Setting pretrained=True loads weights trained on ImageNet
model = torchvision.models.efficientnet_b0(pretrained=True)

# freeze all parameters in the model so that their gradients are not computed
# this prevents updates to these parameters during training, allowing us to use the pre-trained features
for param in model.parameters():
    param.requires_grad = False


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 88.6MB/s]


In [ ]:
# replace the final classification layer of the EfficientNet-B0 model to match the number of classes
# in the Flowers102 dataset (102 classes), this ensures the model can output predictions for each class
model.classifier[1] = nn.Linear(
    in_features=model.classifier[1].in_features,  # number of input features from the pre-trained model
    out_features=102                              # number of output classes in the Flowers102 dataset
)

# enable gradient computation for the parameters in the classifier layer
# this allows the final layer to be trained to adapt to the new dataset (Flowers102)
for param in model.classifier.parameters():
    param.requires_grad = True


In [ ]:
# check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# move the model to the selected device (GPU or CPU) to ensure that all computations are performed on that device
model = model.to(device)


In [ ]:
# define the loss function as CrossEntropyLoss, which is commonly used for multi-class classification tasks
# this loss function measures the difference between the model's predictions and the true labels
criterion = nn.CrossEntropyLoss()

# define the optimizer as Adam, which is an algorithm that adjusts the learning rate
# for each parameter. Only parameters with requires_grad=True (i.e., those not frozen) will be updated
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),  # filter parameters to update only trainable ones
    lr=0.001                                                # set the learning rate to 0.001
)


In [ ]:
def train_model(model, dataloader, criterion, optimizer, device):
    # set the model to training mode, enabling layers like dropout and batch normalization
    model.train()
    
    # initialize variables to accumulate the total loss and count correct predictions
    running_loss = 0.0
    correct = 0
    total = 0
    
    # iterate over each batch of inputs and labels in the dataloader
    for inputs, labels in dataloader:
        # move the inputs and labels to the specified device 
        inputs, labels = inputs.to(device), labels.to(device)
        
        # zero the gradients for the optimizer to prepare for the next gradient computation
        optimizer.zero_grad()
        
        # perform a forward pass through the model to get predictions
        outputs = model(inputs)
        
        # calculate the loss between the predictions and true labels
        loss = criterion(outputs, labels)
        
        # perform a backward pass to compute gradients for each parameter
        loss.backward()
        
        # update the model's parameters based on the gradients computed
        optimizer.step()
        
        # accumulate the loss for this batch to calculate the average loss later
        running_loss += loss.item()
        
        # get the predicted class by finding the index of the maximum logit
        _, predicted = torch.max(outputs, 1)
        
        # update the total count of labels and the count of correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    # return the average loss and accuracy over the entire dataset
    return running_loss / len(dataloader), correct / total


In [ ]:
def validate_model(model, dataloader, criterion, device):
    # set the model to evaluation mode to disable layers like dropout and batch normalization
    model.eval()
    
    # initialize variables to accumulate the total loss and count correct predictions
    running_loss = 0.0
    correct = 0
    total = 0
    
    # disable gradient computation for validation 
    with torch.no_grad():
        # iterate over each batch of inputs and labels in the dataloader
        for inputs, labels in dataloader:
            # move the inputs and labels to the specified device 
            inputs, labels = inputs.to(device), labels.to(device)
            
            # perform a forward pass through the model to get predictions
            outputs = model(inputs)
            
            # calculate the loss between the predictions and true labels
            loss = criterion(outputs, labels)
            
            # accumulate the loss for this batch to calculate the average loss later
            running_loss += loss.item()
            
            # get the predicted class by finding the index of the maximum logit
            _, predicted = torch.max(outputs, 1)
            
            # update the total count of labels and the count of correct predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    # return the average loss and accuracy over the entire validation dataset
    return running_loss / len(dataloader), correct / total


In [ ]:
def test_model(model, dataloader, device):
    # set the model to evaluation mode to disable layers like dropout and batch normalization
    model.eval()
    
    # initialize variables to track the number of correct predictions and total samples
    correct = 0
    total = 0
    
    # disable gradient computation for testing 
    with torch.no_grad():
        # iterate over each batch of inputs and labels in the dataloader
        for inputs, labels in dataloader:
            # move the inputs and labels to the specified device (gpu or cpu)
            inputs, labels = inputs.to(device), labels.to(device)
            
            # perform a forward pass through the model to get predictions
            outputs = model(inputs)
            
            # get the predicted class by finding the index of the maximum logit
            _, predicted = torch.max(outputs, 1)
            
            # update the total count of labels and the count of correct predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    # return the accuracy over the entire test dataset
    return correct / total


In [ ]:
# set the number of epochs for training
num_epochs = 100
# initialize the best validation accuracy to track the highest performance
best_val_acc = 0.0
# set the patience parameter for early stopping (how many epochs to wait for improvement)
patience = 5
# initialize a counter to track epochs with no improvement in validation accuracy
counter = 0

# lists to store the history of train and validation losses and accuracies
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []

# loop over the number of epochs for training
for epoch in range(num_epochs):
    # train the model on the training set and get the loss and accuracy
    train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, device)
    
    # validate the model on the validation set and get the loss and accuracy
    val_loss, val_acc = validate_model(model, val_loader, criterion, device)
    
    # append the results for this epoch to the history lists
    train_loss_history.append(train_loss)
    train_acc_history.append(train_acc)
    val_loss_history.append(val_loss)
    val_acc_history.append(val_acc)
    
    # print the loss and accuracy for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

    # check if the validation accuracy has improved
    if val_acc > best_val_acc:
        # update the best validation accuracy
        best_val_acc = val_acc
        # save the model with the best validation accuracy
        torch.save(model.state_dict(), 'Best_EfficientNetb0_FullFineTuning.pth')
        # reset the counter as we have seen an improvement
        counter = 0
    else:
        # increment the counter when no improvement is observed
        counter += 1

    # check if early stopping criteria are met
    if counter >= patience:
        # print a message indicating early stopping due to lack of improvement
        print(f"Early stopping at epoch {epoch+1}. No improvement in validation accuracy for {patience} consecutive epochs.")
        break


Epoch [1/100], Train Loss: 4.3415, Train Accuracy: 0.1353, Validation Loss: 3.5982, Validation Accuracy: 0.5176
Epoch [2/100], Train Loss: 2.9215, Train Accuracy: 0.7931, Validation Loss: 2.7691, Validation Accuracy: 0.7049
Epoch [3/100], Train Loss: 2.0210, Train Accuracy: 0.9000, Validation Loss: 2.1987, Validation Accuracy: 0.7608
Epoch [4/100], Train Loss: 1.3850, Train Accuracy: 0.9569, Validation Loss: 1.8535, Validation Accuracy: 0.7941
Epoch [5/100], Train Loss: 0.9946, Train Accuracy: 0.9618, Validation Loss: 1.6125, Validation Accuracy: 0.8078
Epoch [6/100], Train Loss: 0.7519, Train Accuracy: 0.9804, Validation Loss: 1.4222, Validation Accuracy: 0.8088
Epoch [7/100], Train Loss: 0.5962, Train Accuracy: 0.9794, Validation Loss: 1.3157, Validation Accuracy: 0.8176
Epoch [8/100], Train Loss: 0.4635, Train Accuracy: 0.9951, Validation Loss: 1.2286, Validation Accuracy: 0.8255
Epoch [9/100], Train Loss: 0.3910, Train Accuracy: 0.9951, Validation Loss: 1.1607, Validation Accuracy:

In [ ]:
# cell for plotting training and validation metrics
import matplotlib.pyplot as plt
epochs = range(1, len(train_loss_history) + 1)

plt.figure(figsize=(10, 5))
plt.subplot(2, 1, 1)
plt.plot(epochs, train_loss_history, 'b-', label='Training Loss')
plt.plot(epochs, val_loss_history, 'orange', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss History')
plt.legend()
plt.grid()


plt.subplot(2, 1, 2)
plt.plot(epochs, train_acc_history, 'b-', label='Training Accuracy')
plt.plot(epochs, val_acc_history, 'orange', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy History')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

In [ ]:
# evaluating best model that was saved on test dataset
model.load_state_dict(torch.load('Best_EfficientNetb0_FullFineTuning.pth'))
test_acc = test_model(model, test_loader, device)
print(f"Test Accuracy: {test_acc:.4f}")

<ipython-input-14-56af308653c7>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('Best_EfficientNetb0_FullFineTuning.pth'))


Test Accuracy: 0.7966


## Analysis of Results
### The training and validation loss plots indicate a steady decrease in both metrics, with training loss dropping more rapidly, suggesting that the model is learning effectively without overfitting. The accuracy plot shows that training accuracy approaches 1.0, while validation accuracy stabilizes around 0.83, indicating a good generalization to unseen data. Early stopping was triggered due to the lack of validation accuracy improvement after several epochs, yielding a final test accuracy of 0.7966, which aligns closely with validation performance.